In [1]:
import supereeg as se
import pandas as pd
import numpy as np
import os

In [2]:
pwd

'/Users/lucyowen/repos/supereeg_paper/code/notebooks'

In [3]:
ram_data_dir = '/Users/lucyowen/repos/supereeg_paper/data/ram'
pyfr_data_dir = '/Users/lucyowen/repos/supereeg_paper/data/pyfr'

In [4]:
pyfr_data_dir

'/Users/lucyowen/repos/supereeg_paper/data/pyfr'

## PyFR analysis

In [5]:
pyfr_across = pd.read_csv('/Users/lucyowen/Desktop/paper_figures/pyfr/pyfr_across.csv', index_col=0)
pyfr_within = pd.read_csv('/Users/lucyowen/Desktop/paper_figures/pyfr/pyfr_within.csv', index_col=0)

In [6]:
pyfr_across.head()

,Unnamed: 0.1,R,Correlation,Subject,Electrode,Sample rate,Samples,Total Electrodes,Sessions,Number thresholded
0,0,[-63.64 -21.93 5.37],0.844742,FR190,7,250.0,3465000,10,7.0,42
1,0,[-48.48 38.4 17.26],0.861268,CH013,34,250.0,1549237,121,2.0,2
2,0,[ 49.49 -34.7 35.17],0.665390,CH048,36,250.0,2454000,44,4.0,79
3,0,[ 65.66 -67.29 25.83],0.641038,TJ019,62,250.0,3600000,82,2.0,16
4,0,[-36.36 -1.81 59.76],0.764453,FR050,51,250.0,447750,98,1.0,2


In [7]:
pyfr_within.head()

,Unnamed: 0.1,R,Correlation,Full,Subject
0,0,[ 61.62 -13.33 -25.8 ],0.310237,TJ032_62_within,TJ032
1,0,[ 63.64 -70.63 31.1 ],0.665180,FR150_52_within,FR150
2,0,[-56.57 0.16 -27.4 ],0.683088,CH068_2_within,CH068
3,0,[-55.56 35.11 21.44],0.371791,CH067_37_within,CH067
4,0,[62.63 -2.89 39.03],0.443610,FR220_28_within,FR220


## Merge dataframes

In [8]:
pyFR_df = pd.merge(pyfr_across, pyfr_within, on=['Subject', 'R'])
pyFR_df.dropna(inplace=True)
pyFR_df.rename(columns={"Correlation_x":"Correlation_Across", "Correlation_y":"Correlation_Within"}, inplace=True)
e_corrs = pd.DataFrame()
e_corrs['across'] = pyFR_df['Correlation_Across']
e_corrs['within'] = pyFR_df['Correlation_Within']
e_corrs['Subject'] = pyFR_df['Subject']
e_corrs['R'] = pyFR_df['R']
e_corrs['Sample rate'] = pyFR_df['Sample rate']
e_corrs['Samples'] = pyFR_df['Samples']
e_corrs['Thresholded Electrodes'] = pyFR_df['Total Electrodes']
e_corrs['Sessions'] = pyFR_df['Sessions']
e_corrs['Number thresholded'] = pyFR_df['Number thresholded']

In [24]:
e_corrs.head()

,across,within,Subject,R,Sample rate,Samples,Thresholded Electrodes,Sessions,Number thresholded,Total Time (hrs),Average Session Length (hrs),Total Electrodes,Proportion Thresholded
0,0.844742,0.639768,31,[-63.64 -21.93 5.37],250.0,3465000,10,7.0,42,3.850,0.550,52,0.808
1,0.861268,0.640264,8,[-48.48 38.4 17.26],250.0,1549237,121,2.0,2,1.721,0.860,123,0.016
2,0.665390,0.519857,13,[ 49.49 -34.7 35.17],250.0,2454000,44,4.0,79,2.727,0.682,123,0.642
3,0.641038,0.445575,52,[ 65.66 -67.29 25.83],250.0,3600000,82,2.0,16,4.000,2.000,98,0.163
4,0.764453,0.504416,25,[-36.36 -1.81 59.76],250.0,447750,98,1.0,2,0.498,0.498,100,0.020


## With a few other stats

In [10]:
e_corrs['Total Time (hrs)'] = np.round(e_corrs['Samples']/e_corrs['Sample rate']/3600,3)
e_corrs['Average Session Length (hrs)'] = np.round(e_corrs['Total Time (hrs)']/e_corrs['Sessions'],3)
e_corrs['Average Session Length (hrs)'].head()
e_corrs['Total Electrodes'] = e_corrs['Thresholded Electrodes'] + e_corrs['Number thresholded']
e_corrs['Proportion Thresholded'] = np.round(e_corrs['Number thresholded'] / e_corrs['Total Electrodes'], 3)

## Rename subject ids 

In [11]:
grouped_results_e_corrs = e_corrs.groupby('Subject')['within', 'across', 'Sample rate', 'Samples', 
                                                  'Sessions', 'Number thresholded', 'Total Time (hrs)', 'Average Session Length (hrs)', 
                                                  'Total Electrodes', 'Proportion Thresholded'].mean()

grouped_results_e_corrs['Subject'] = range(grouped_results_e_corrs.shape[0])
subj_dict = grouped_results_e_corrs['Subject'].to_dict()
e_corrs.Subject = [subj_dict[item] for item in e_corrs.Subject] 

In [12]:
e_corrs.to_csv(os.path.join(pyfr_data_dir, 'pyfr.csv'))

# RAM

In [13]:
W_a_a = pd.read_csv('/Users/lucyowen/Desktop/paper_figures/ram/all/a_sub-a_exp.csv', index_col=0)
W_a_a.name = 'W_a_a'
W_a_all = pd.read_csv('/Users/lucyowen/Desktop/paper_figures/ram/all/a_sub-all_exp.csv', index_col=0)
W_a_all.name = 'W_a_all'
W_a_w = pd.read_csv('/Users/lucyowen/Desktop/paper_figures/ram/all/a_sub-w_exp.csv', index_col=0)
W_a_w.name = 'W_a_w'
W_w_a = pd.read_csv('/Users/lucyowen/Desktop/paper_figures/ram/all/w_sub-a_exp.csv', index_col=0)
W_w_a.name = 'W_w_a'
W_w_all = pd.read_csv('/Users/lucyowen/Desktop/paper_figures/ram/all/w_sub-all_exp.csv', index_col=0)
W_w_all.name = 'W_w_all'
W_w_w = pd.read_csv('/Users/lucyowen/Desktop/paper_figures/ram/all/w_sub-w_exp.csv', index_col=0)
W_w_w.name = 'W_w_w'

In [14]:
dfs = [W_a_a, W_a_all, W_a_w, W_w_a, W_w_all, W_w_w]

In [15]:
RAM_df = dfs[0]
RAM_df = RAM_df.rename(columns={"Correlation":RAM_df.name})
for df_ in dfs[1:]:
    RAM_df = RAM_df.merge(df_, on=['Subject', 'R', 'Electrode', 'Samples', 'Total Electrodes','Sample rate','Sessions', 'Number thresholded'])
    
    RAM_df = RAM_df.rename(columns={"Correlation":df_.name})

In [16]:
RAM_df.dropna(inplace=True)

In [17]:
RAM_df['Sessions'] = RAM_df['Sessions'] + 1

In [18]:
RAM_df['Thresholded Electrodes'] = RAM_df['Total Electrodes']
RAM_df['Total Time (hrs)'] = np.round(RAM_df['Samples']/RAM_df['Sample rate']/3600,3)
RAM_df['Average Session Length (hrs)'] = np.round(RAM_df['Total Time (hrs)']/RAM_df['Sessions'],3)
RAM_df['Average Session Length (hrs)'].head()
RAM_df['Total Electrodes'] = RAM_df['Thresholded Electrodes'] + RAM_df['Number thresholded']
RAM_df['Proportion Thresholded'] = np.round(RAM_df['Number thresholded'] / RAM_df['Total Electrodes'], 3)

In [19]:
all_subject_e = np.array([])
all_exp_e = np.array([])
for i , item in enumerate(RAM_df['Subject'].values):
    all_subject_e = np.append(all_subject_e, item.split('_')[0])
    all_exp_e = np.append(all_exp_e, item.split('_')[1]+ '_' + item.split('_')[2])

In [20]:
RAM_df['Subject'] = all_subject_e
RAM_df['Experiment'] = all_exp_e

In [21]:
grouped_results_ram = RAM_df.groupby('Subject')['Experiment','W_w_w', 'W_w_a', 'W_w_all', 'W_a_w', 'W_a_a','W_a_all', 'Sample rate', 'Samples', 
                                                  'Sessions', 'Number thresholded', 'Total Time (hrs)', 'Average Session Length (hrs)', 
                                                  'Total Electrodes', 'Proportion Thresholded'].mean()

grouped_results_ram['Subject'] = range(grouped_results_ram.shape[0])
subj_dict = grouped_results_ram['Subject'].to_dict()
RAM_df.Subject = [subj_dict[item] for item in RAM_df.Subject] 

In [22]:
RAM_df

,R,W_a_a,Subject,Electrode,Sample rate,Samples,Total Electrodes,Sessions,Number thresholded,W_a_all,W_a_w,W_w_a,W_w_all,W_w_w,Thresholded Electrodes,Total Time (hrs),Average Session Length (hrs),Proportion Thresholded,Experiment
0,[-50.39 18.22 -43.3 ],0.242494,3,2,250.0,588250,86,1.0,14,0.300987,0.447637,-0.162586,0.236228,0.061755,72,0.654,0.654,0.163,RAM_CatFR1
1,[-46.17 18.21 -42.23],0.177056,3,1,250.0,588250,86,1.0,14,0.046046,0.478972,0.165845,0.275166,0.292074,72,0.654,0.654,0.163,RAM_CatFR1
2,[-57.19 19.07 -44.75],0.234188,3,4,250.0,588250,86,1.0,14,0.788364,0.916196,0.542039,-0.021315,0.117429,72,0.654,0.654,0.163,RAM_CatFR1
3,[-43.21 18.21 -41.49],0.336156,3,0,250.0,588250,86,1.0,14,0.536711,0.796827,0.005241,-0.211179,0.046857,72,0.654,0.654,0.163,RAM_CatFR1
4,[-54.86 15.87 40.38],0.024989,0,3,250.0,668750,104,1.0,19,0.426785,0.232029,0.187670,0.312530,0.378536,85,0.743,0.743,0.183,RAM_FR1
5,[-38.79 51.43 47.05],0.049455,0,0,250.0,668750,104,1.0,19,0.191558,0.514627,0.113440,0.072644,0.461210,85,0.743,0.743,0.183,RAM_FR1
6,[-52.44 24.4 43.61],0.104537,0,2,250.0,668750,104,1.0,19,0.294488,0.303376,0.284694,0.395410,0.345201,85,0.743,0.743,0.183,RAM_FR1
7,[-53.39 19.07 -43.78],0.135624,3,3,250.0,588250,86,1.0,14,0.567127,0.097120,0.491727,0.073085,0.043775,72,0.654,0.654,0.163,RAM_CatFR1
8,[-38.79 51.43 47.05],0.234099,0,0,250.0,945000,104,2.0,24,0.537998,0.637745,0.137081,0.295410,0.270472,80,1.050,0.525,0.231,RAM_CatFR1
9,[-43.98 42.16 46.89],-0.045409,0,1,250.0,945000,104,2.0,24,0.647666,0.710633,0.440786,0.465773,0.319532,80,1.050,0.525,0.231,RAM_CatFR1


In [23]:
RAM_df.to_csv(os.path.join(ram_data_dir, 'ram.csv'))